In [58]:
from google.colab import files
from google.colab import drive 
drive.mount("/drive")
uploaded = files.upload()

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


Saving AAPL_new.csv to AAPL_new (1).csv
Saving data_AAPL.csv to data_AAPL (1).csv


# 1. basic regression

In [ ]:
#  pip install mpi4py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=4438382 sha256=e1d82ed244905bf4afcb95fedd7c19e43b6700b2b7b0e70764fa76e8f96588c9
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py


In [59]:
! pip install gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
 pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
import pandas as pd
# from numba import jit, cuda
# from mpi4py import MPI
import numba
appl_price=pd.read_csv("AAPL_new.csv")
appl_tw=pd.read_csv('data_AAPL.csv',lineterminator='\n')

In [43]:
# comm = MPI.COMM_WORLD 
# rank = comm.Get_rank()
# size = comm.Get_size()


In [107]:
appl_price['Date']=pd.to_datetime(appl_price['Date'],format='%Y.%m.%d')
appl_price.rename(columns={'Unnamed: 0':'ID'},inplace = True)
appl_price.info()
appl_price

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3790 entries, 0 to 3789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            3790 non-null   int64         
 1   Date          3790 non-null   datetime64[ns]
 2   Open          3790 non-null   float64       
 3   High          3790 non-null   float64       
 4   Low           3790 non-null   float64       
 5   Close         3790 non-null   float64       
 6   Adj Close     3790 non-null   float64       
 7   Volume        3790 non-null   int64         
 8   Normalized    3790 non-null   float64       
 9   Standardized  3790 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(2)
memory usage: 296.2 KB


,ID,Date,Open,High,Low,Close,Adj Close,Volume,Normalized,Standardized
0,0,2007-11-01,6.735714,6.789286,6.428571,6.694286,5.706328,805036400,0.018623,-0.788990
1,1,2007-11-02,6.757500,6.765714,6.553214,6.709643,5.719419,1002114400,0.018696,-0.788708
2,2,2007-11-05,6.617500,6.748571,6.580000,6.649286,5.667971,804176800,0.018408,-0.789816
3,3,2007-11-06,6.680357,6.857143,6.616786,6.849643,5.838758,954727200,0.019364,-0.786136
4,4,2007-11-07,6.807500,6.881429,6.647500,6.653571,5.671623,994327600,0.018428,-0.789738
...,...,...,...,...,...,...,...,...,...,...
3785,3785,2022-11-14,148.970001,150.279999,147.429993,148.279999,148.279999,73374100,0.817001,2.283482
3786,3786,2022-11-15,152.220001,153.589996,148.559998,150.039993,150.039993,89868300,0.826857,2.321410
3787,3787,2022-11-16,149.130005,149.869995,147.289993,148.789993,148.789993,64218300,0.819857,2.294472
3788,3788,2022-11-17,146.429993,151.479996,146.149994,150.720001,150.720001,80389400,0.830665,2.336064


In [108]:
appl_tw.rename(columns={'Created_at':'Date','Unnamed: 0':'ID'},inplace = True)
appl_tw['Date']=pd.to_datetime(appl_tw['Date']).dt.date  #.view('int64')#.floor('d').view('int64')
appl_tw['Date']=pd.to_datetime(appl_tw['Date'],format='%Y.%m.%d')
appl_tw.info()
appl_tw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99506 entries, 0 to 99505
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ID      99506 non-null  int64         
 1   User    99506 non-null  float64       
 2   Date    99506 non-null  datetime64[ns]
 3   Tweet   99506 non-null  object        
 4   Tokens  99506 non-null  object        
 5   Score   99506 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ MB


,ID,User,Date,Tweet,Tokens,Score
0,0,5.555773e+08,2008-01-02,Is anyone going to Mac World San Fran? #apple ...,"[""anyone"", ""going"", ""mac"", ""world"", ""san"", ""fr...",0
1,1,5.614809e+08,2008-01-04,#Apple How addicted to Apple are you? http://t...,"[""addicted"", ""apple""]",0
2,2,5.643486e+08,2008-01-05,I'm selling my #Apple #MacBook #laptop #notebo...,"[""selling""]",0
3,3,5.671331e+08,2008-01-06,I heartz my ipod. :) #ipod #apple,"[""heartz"", ""ipod""]",0
4,4,5.703977e+08,2008-01-07,#Apple keeping #Intel in the dark on Mobile Ap...,"[""keeping"", ""dark"", ""mobile"", ""apple"", ""device...",0
...,...,...,...,...,...,...
99501,99501,1.594295e+18,2022-11-20,Why not check out the latest episode of Roll B...,"[""check"", ""latest"", ""episode"", ""roll"", ""britan...",0
99502,99502,1.594294e+18,2022-11-20,#Xiaomi Mi Air Purifier 3 to be available for ...,"[""mi"", ""air"", ""purifier"", ""available"", ""purcha...",0
99503,99503,1.594293e+18,2022-11-20,pple Canvas Print Kitchen &amp; Dining Decor W...,"[""pple"", ""canvas"", ""print"", ""kitchen"", ""dining...",1
99504,99504,1.594293e+18,2022-11-20,Apple To Bring Proper Battery-Saving Mode to W...,"[""apple"", ""bring"", ""proper"", ""mode"", ""watchos""...",1


In [129]:
appl_mergeDp=pd.merge(appl_price,appl_tw,left_on='Date', right_on='Date')
#appl_predict_pd=appl_mergeDp.drop(columns=['ID_x','Open','High','Low','Close','Volume','Normalized','Standardized','ID_y','User','Tweet','Tokens'])
#appl_predict_pd
#display(appl_mergeDp)
appl_predict_pd=appl_mergeDp.drop(columns=['ID_x','ID_y','User','Tweet','Tokens','Date','High','Low'])
#appl_predict_pd=appl_mergeDp.drop(columns=['ID_x','ID_y','User','Date'])
appl_predict_pd

breakTime=appl_predict_pd['Open'][0:]
display(type(breakTime))
#breakTime.append(0.0)
display(breakTime)
for i in range(0,breakTime.size-1):
    breakTime.iloc[i]=breakTime.iloc[i+1]
appl_predict_pd['Price']=appl_predict_pd['Open'][0:]
appl_predict_pd


# @numba.jit(parallel=True)
# def get_breaktime(predic_df):
#   breakTime=predic_df
#   # i = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
#   display(type(breakTime))
#   #breakTime.append(0.0)
#   display(breakTime)
#   for i in range(0,breakTime.size-1):
#       breakTime.iloc[i]=breakTime.iloc[i+1] 
     
# appl_predict_pd['Price']=get_breaktime(appl_predict_pd['Open'][0:])
# appl_predict_pd


pandas.core.series.Series

0          7.116786
1          6.837500
2          6.473214
3          6.473214
4          6.473214
            ...    
68865    152.309998
68866    152.309998
68867    152.309998
68868    152.309998
68869    152.309998
Name: Open, Length: 68870, dtype: float64

,Open,Close,Adj Close,Volume,Normalized,Standardized,Score,Price
0,6.837500,6.958571,5.931611,1079178800,0.019884,-0.784135,0,6.837500
1,6.473214,6.430357,5.481352,1455832000,0.017363,-0.793838,0,6.473214
2,6.473214,6.344286,5.407982,2072193200,0.016952,-0.795419,0,6.473214
3,6.473214,6.344286,5.407982,2072193200,0.016952,-0.795419,1,6.473214
4,6.473214,6.344286,5.407982,2072193200,0.016952,-0.795419,-1,6.473214
...,...,...,...,...,...,...,...,...
68865,152.309998,151.289993,151.289993,74794600,0.833857,2.348347,1,152.309998
68866,152.309998,151.289993,151.289993,74794600,0.833857,2.348347,0,152.309998
68867,152.309998,151.289993,151.289993,74794600,0.833857,2.348347,0,152.309998
68868,152.309998,151.289993,151.289993,74794600,0.833857,2.348347,0,152.309998


In [126]:
appl_predict_pd['Price']

0        None
1        None
2        None
3        None
4        None
         ... 
68865    None
68866    None
68867    None
68868    None
68869    None
Name: Price, Length: 68870, dtype: object

In [122]:
appl_mergeDp['Price']=breakTime
appl_mergeDp

,ID_x,Date,Open,High,Low,Close,Adj Close,Volume,Normalized,Standardized,ID_y,User,Tweet,Tokens,Score,Price
0,41,2008-01-02,7.116786,7.152143,6.876786,6.958571,5.931611,1079178800,0.019884,-0.784135,0,5.555773e+08,Is anyone going to Mac World San Fran? #apple ...,"[""anyone"", ""going"", ""mac"", ""world"", ""san"", ""fr...",0,6.837500
1,43,2008-01-04,6.837500,6.892857,6.388929,6.430357,5.481352,1455832000,0.017363,-0.793838,1,5.614809e+08,#Apple How addicted to Apple are you? http://t...,"[""addicted"", ""apple""]",0,6.473214
2,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,4,5.703977e+08,#Apple keeping #Intel in the dark on Mobile Ap...,"[""keeping"", ""dark"", ""mobile"", ""apple"", ""device...",0,6.473214
3,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,5,5.703794e+08,@thattalldude same for the rest of the mac co...,"[""rest"", ""mac"", ""community"", ""want"", ""made"", ""...",1,6.473214
4,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,6,5.700020e+08,I wonder how much #Apple will pay Comwave to u...,"[""wonder"", ""much"", ""pay"", ""comwave"", ""use"", ""t...",-1,6.473214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68865,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99461,1.593572e+18,"iOS 16 is a great upgrade, but there are a few...","[""ios"", ""great"", ""upgrade"", ""features"", ""like""...",1,152.309998
68866,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99462,1.593572e+18,#Vivo X90 Series may launch this month\nRead m...,"[""series"", ""may"", ""launch"", ""month"", ""read""]",0,152.309998
68867,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99463,1.593572e+18,All your #Podcast Links #Apple #Tunein #iHear...,"[""links""]",0,152.309998
68868,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99464,1.593572e+18,#Apple (@Apple) Music : 3 mois offerts pour le...,"[""music"", ""mois"", ""offerts"", ""pour"", ""les"", ""a...",0,152.309998


In [ ]:
#print(tf.__version__)
# dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
# dataset_path
# column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
#                 'Acceleration', 'Model Year', 'Origin']
# raw_dataset = pd.read_csv(dataset_path, names=column_names,
#                       na_values = "?", comment='\t',
#                       sep=" ", skipinitialspace=True)

# dataset = raw_dataset.copy()
# dataset.isna().sum()
# dataset = dataset.dropna()
# origin = dataset.pop('Origin')
# dataset['USA'] = (origin == 1)*1.0
# dataset['Europe'] = (origin == 2)*1.0
# dataset['Japan'] = (origin == 3)*1.0
# dataset.tail()
# train_dataset = dataset.sample(frac=0.8,random_state=0)
# test_dataset = dataset.drop(train_dataset.index)
# train_stats = train_dataset.describe()
# train_stats.pop("MPG")
# train_stats = train_stats.transpose()
# train_stats
# def norm(x):
#     return (x - train_stats['mean']) / train_stats['std']
# normed_train_data = norm(train_dataset)
# normed_test_data = norm(test_dataset)

#display(train_dataset.keys())
#display(train_dataset)

In [125]:
y=appl_predict_pd[['Price']]
X=appl_predict_pd[['Open','Close','Adj Close','Normalized','Standardized','Score']]
display(X)
y

,Open,Close,Adj Close,Normalized,Standardized,Score
0,6.837500,6.958571,5.931611,0.019884,-0.784135,0
1,6.473214,6.430357,5.481352,0.017363,-0.793838,0
2,6.473214,6.344286,5.407982,0.016952,-0.795419,0
3,6.473214,6.344286,5.407982,0.016952,-0.795419,1
4,6.473214,6.344286,5.407982,0.016952,-0.795419,-1
...,...,...,...,...,...,...
68865,152.309998,151.289993,151.289993,0.833857,2.348347,1
68866,152.309998,151.289993,151.289993,0.833857,2.348347,0
68867,152.309998,151.289993,151.289993,0.833857,2.348347,0
68868,152.309998,151.289993,151.289993,0.833857,2.348347,0


0        None
1        None
2        None
3        None
4        None
         ... 
68865    None
68866    None
68867    None
68868    None
68869    None
Name: Price, Length: 68870, dtype: object

In [119]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

             Open       Close   Adj Close  Normalized  Standardized  Score
54156   64.332497   64.375000   62.877968    0.338770      0.443063      1
20730   16.211071   15.797143   13.585352    0.062743     -0.619196      0
5178     6.762857    6.781071    5.780306    0.019037     -0.787395      1
56536   71.182503   71.932503   70.637909    0.382224      0.610290      1
59810  135.580002  133.720001  132.133820    0.726587      1.935531      0
...           ...         ...         ...         ...           ...    ...
21243   15.301786   15.298214   13.232955    0.060770     -0.626790      0
45891   43.802502   44.205002   42.205906    0.223012     -0.002422      1
42613   36.112499   36.005001   33.981071    0.176955     -0.179667      0
43567   40.125000   40.375000   38.255054    0.200888     -0.087563      0
68268  142.539993  140.089996  139.857986    0.769840      2.101987      1

[55096 rows x 6 columns]
            Price
54156   64.332497
20730   16.211071
5178     6.762857
56

In [134]:
#regrssion
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

@numba.jit(parallel=True,forceobj = True)
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[6]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

model = build_model()
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS =10
model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


..........

In [136]:
from sklearn.metrics import mean_squared_error

test_predictions = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, test_predictions)
print('mse={0}'.format(mse))
predict_df = pd.DataFrame(test_predictions)

display(predict_df)

431/431 [==============================] - 1s 2ms/step
mse=5.075520692992165


,0
0,150.150604
1,8.881479
2,45.700325
3,39.604713
4,24.838163
...,...
13769,57.319241
13770,41.232121
13771,173.799301
13772,24.753105


# 2. add volume

In [137]:
y=appl_predict_pd[['Price']]
X=appl_predict_pd[['Open','Close','Adj Close','Normalized','Standardized','Score','Volume']]
print(y.shape, X.shape)

(68870, 1) (68870, 7)


In [138]:
X

,Open,Close,Adj Close,Normalized,Standardized,Score,Volume
0,6.837500,6.958571,5.931611,0.019884,-0.784135,0,1079178800
1,6.473214,6.430357,5.481352,0.017363,-0.793838,0,1455832000
2,6.473214,6.344286,5.407982,0.016952,-0.795419,0,2072193200
3,6.473214,6.344286,5.407982,0.016952,-0.795419,1,2072193200
4,6.473214,6.344286,5.407982,0.016952,-0.795419,-1,2072193200
...,...,...,...,...,...,...,...
68865,152.309998,151.289993,151.289993,0.833857,2.348347,1,74794600
68866,152.309998,151.289993,151.289993,0.833857,2.348347,0,74794600
68867,152.309998,151.289993,151.289993,0.833857,2.348347,0,74794600
68868,152.309998,151.289993,151.289993,0.833857,2.348347,0,74794600


In [139]:
# Normalization
# ref: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X

array([[-0.85754685, -0.85477539, -0.83242665, ..., -0.83242665,
        -0.36576398,  3.09830955],
       [-0.86544011, -0.86621184, -0.84210551, ..., -0.84210551,
        -0.36576398,  4.57519214],
       [-0.86544011, -0.86807538, -0.84368268, ..., -0.84368268,
        -0.36576398,  6.99198595],
       ...,
       [ 2.29451598,  2.27016862,  2.29222577, ...,  2.29222577,
        -0.36576398, -0.8399486 ],
       [ 2.29451598,  2.27016862,  2.29222577, ...,  2.29222577,
        -0.36576398, -0.8399486 ],
       [ 2.29451598,  2.27016862,  2.29222577, ...,  2.29222577,
         1.10751741, -0.8399486 ]])

In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(55096, 7) (13774, 7) (55096, 1) (13774, 1)


In [142]:
#regression
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

@numba.jit(parallel=True,forceobj = True)
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[7]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model
model = build_model()
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS = 10
model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1,
  callbacks=[PrintDot()])

Epoch 1/10
1378/1378 [==============================] - 5s 3ms/step - loss: 242.4144 - mae: 5.1492 - mse: 242.4144 - val_loss: 0.7309 - val_mae: 0.4622 - val_mse: 0.7309
Epoch 2/10
1378/1378 [==============================] - 4s 3ms/step - loss: 0.9280 - mae: 0.5425 - mse: 0.9280 - val_loss: 0.6311 - val_mae: 0.4735 - val_mse: 0.6311
Epoch 3/10
1378/1378 [==============================] - 3s 3ms/step - loss: 0.7940 - mae: 0.4941 - mse: 0.7940 - val_loss: 0.8227 - val_mae: 0.4730 - val_mse: 0.8227
Epoch 4/10
1378/1378 [==============================] - 3s 3ms/step - loss: 0.6745 - mae: 0.4562 - mse: 0.6745 - val_loss: 0.3491 - val_mae: 0.3150 - val_mse: 0.3491
Epoch 5/10
1378/1378 [==============================] - 4s 3ms/step - loss: 0.6017 - mae: 0.4259 - mse: 0.6017 - val_loss: 0.4950 - val_mae: 0.4936 - val_mse: 0.4950
Epoch 6/10
1378/1378 [==============================] - 4s 3ms/step - loss: 0.5278 - mae: 0.4036 - mse: 0.5278 - val_loss: 0.8332 - val_mae: 0.4734 - val_mse: 0.8332


In [143]:
from sklearn.metrics import mean_squared_error

test_predictions = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, test_predictions)
print('mse={0}'.format(mse))
predict_df = pd.DataFrame(test_predictions)

display(predict_df)

431/431 [==============================] - 1s 2ms/step
mse=0.9275301338976938


,0
0,143.244843
1,9.437819
2,44.039841
3,38.316654
4,24.139685
...,...
13769,55.023518
13770,39.909870
13771,165.411407
13772,24.384289


In [144]:
y_test

,Price
62483,144.809998
6804,8.587143
51991,44.105000
43820,38.472500
28312,24.037500
...,...
49012,55.575001
43355,40.107498
67452,167.679993
27883,23.965000


# 3. add Tokens

- input:['Open','Close','Adj Close','Normalized','Standardized','Score','Volume', 'Tokens']
- output: price


## 3.1 token statistic

In [145]:
appl_mergeDp

,ID_x,Date,Open,High,Low,Close,Adj Close,Volume,Normalized,Standardized,ID_y,User,Tweet,Tokens,Score
0,41,2008-01-02,7.116786,7.152143,6.876786,6.958571,5.931611,1079178800,0.019884,-0.784135,0,5.555773e+08,Is anyone going to Mac World San Fran? #apple ...,"[""anyone"", ""going"", ""mac"", ""world"", ""san"", ""fr...",0
1,43,2008-01-04,6.837500,6.892857,6.388929,6.430357,5.481352,1455832000,0.017363,-0.793838,1,5.614809e+08,#Apple How addicted to Apple are you? http://t...,"[""addicted"", ""apple""]",0
2,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,4,5.703977e+08,#Apple keeping #Intel in the dark on Mobile Ap...,"[""keeping"", ""dark"", ""mobile"", ""apple"", ""device...",0
3,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,5,5.703794e+08,@thattalldude same for the rest of the mac co...,"[""rest"", ""mac"", ""community"", ""want"", ""made"", ""...",1
4,44,2008-01-07,6.473214,6.557143,6.079643,6.344286,5.407982,2072193200,0.016952,-0.795419,6,5.700020e+08,I wonder how much #Apple will pay Comwave to u...,"[""wonder"", ""much"", ""pay"", ""comwave"", ""use"", ""t...",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68865,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99461,1.593572e+18,"iOS 16 is a great upgrade, but there are a few...","[""ios"", ""great"", ""upgrade"", ""features"", ""like""...",1
68866,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99462,1.593572e+18,#Vivo X90 Series may launch this month\nRead m...,"[""series"", ""may"", ""launch"", ""month"", ""read""]",0
68867,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99463,1.593572e+18,All your #Podcast Links #Apple #Tunein #iHear...,"[""links""]",0
68868,3789,2022-11-18,152.309998,152.699997,149.970001,151.289993,151.289993,74794600,0.833857,2.348347,99464,1.593572e+18,#Apple (@Apple) Music : 3 mois offerts pour le...,"[""music"", ""mois"", ""offerts"", ""pour"", ""les"", ""a...",0


In [146]:
tokens_set = set([])
tokens_list = []

for token_list in appl_mergeDp["Tokens"].to_list():
    token_list = token_list.replace('[','').replace(']','').replace('"','').split(",")
    token_list = [t.strip() for t in token_list if t!='']
    tokens_list.append(token_list)
    for token in token_list:
        token = token.strip()
        if token!='':
            tokens_set.add(token)

print('nodup token count = {0}'.format(len(tokens_set)))

nodup token count = 33720


In [148]:
tokens_list[:5]

[['anyone', 'going', 'mac', 'world', 'san', 'fran'],
 ['addicted', 'apple'],
 ['keeping', 'dark', 'mobile', 'apple', 'device', 'probably'],
 ['rest',
  'mac',
  'community',
  'want',
  'made',
  'mac',
  'tablet',
  'touchscreen',
  'like',
  'full',
  'os',
  'x',
  'features'],
 ['wonder', 'much', 'pay', 'comwave', 'use', 'trademark', 'canada']]

In [149]:
list(tokens_set)[:200]

['supersport',
 'inappropriate',
 'platiforms',
 'grapple',
 'isnort',
 'proff',
 'lingraphica',
 'sinon',
 'cardium',
 'robinson',
 'pointless',
 'aperture',
 'vapor',
 'implements',
 'constipation',
 'mean',
 'escribe',
 'violent',
 'leaky',
 'pty',
 'amnesty',
 'adware',
 'fin',
 'deutsh',
 'upstart',
 'resisting',
 'stevejewbs',
 'implosion',
 'galsses',
 'kicking',
 'attentats',
 'extremetech',
 'hanks',
 'anderson',
 'starships',
 'strides',
 'commissioner',
 'funeral',
 'deploy',
 'websitespark',
 'rough',
 'confessions',
 'lift',
 'examples',
 'plush',
 'jaye',
 'nerdz',
 'ranch',
 'shouts',
 'expos',
 'buynow',
 'al',
 'pros',
 'enormity',
 'comparisons',
 'myoung',
 'sadness',
 'flawlessly',
 'hardwood',
 'shimmr',
 'tyff',
 'recco',
 'implies',
 'freefall',
 'outlet',
 'nanobyte',
 'traditionalist',
 'fabolous',
 'chamfered',
 'residence',
 'invasion',
 'riot',
 'oxford',
 'surfer',
 'thingmagic',
 'kombucha',
 'kj',
 'repo',
 'significantly',
 'usain',
 'turbo',
 'limiting'

## 3.2. train word2vec model by tokens

In [ ]:
# ! pip install gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [150]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

vsize = 50

# train word2vec model
model = Word2Vec(sentences=tokens_list, vector_size=vsize, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")


In [151]:
# test
vector = model.wv['oldies']
vector.shape

(50,)

## 3.3 tokens to vector

In [152]:
# max token count

token_len_list = []
for t in tokens_list:
    token_len_list.append(len(t))
    
max_token_count = max(token_len_list)
print(max_token_count)

35


In [ ]:
# ! pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [153]:
from tqdm import *

sentence_vector_list = []
for tokens in tqdm(tokens_list):
    sentence_vector = []
    for w in tokens:
        vector = model.wv[w]
        sentence_vector = sentence_vector+list(vector)
    sentence_vector = sentence_vector + [0]*(max_token_count-len(sentence_vector)//vsize)*(vsize)
    sentence_vector_list.append(sentence_vector)    

print(len(sentence_vector_list))

100%|██████████| 68870/68870 [00:22<00:00, 3093.00it/s]

68870


In [154]:
len(sentence_vector)

1750

In [155]:
y=appl_predict_pd[['Price']]
X=appl_predict_pd[['Open','Close','Adj Close','Normalized','Standardized','Score','Volume']]
print(y.shape, X.shape)

(68870, 1) (68870, 7)


In [156]:
import numpy as np

x_data = np.array(X)
y_data = np.array(y)
print('before, x_data.shape = {0}'.format(x_data.shape) )
x_data = np.hstack((x_data, sentence_vector_list))
print('after, x_data.shape = {0}'.format(x_data.shape) )

before, x_data.shape = (68870, 7)
after, x_data.shape = (68870, 1757)


In [157]:
# Normalization
# ref: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data.shape

(68870, 1757)

In [158]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(55096, 1757) (13774, 1757) (55096, 1) (13774, 1)


In [159]:
#回归
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

@numba.jit(parallel=True,forceobj = True)
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_data.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model
model = build_model()
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS = 10
model.fit(
  X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1,
  callbacks=[PrintDot()])

Epoch 1/10
1378/1378 [==============================] - 9s 6ms/step - loss: 284.7269 - mae: 7.2748 - mse: 284.7269 - val_loss: 39.4836 - val_mae: 3.7817 - val_mse: 39.4836
Epoch 2/10
1378/1378 [==============================] - 8s 6ms/step - loss: 28.2338 - mae: 2.4774 - mse: 28.2338 - val_loss: 24.9271 - val_mae: 2.3156 - val_mse: 24.9271
Epoch 3/10
1378/1378 [==============================] - 9s 7ms/step - loss: 19.1130 - mae: 1.9773 - mse: 19.1130 - val_loss: 19.7396 - val_mae: 2.0509 - val_mse: 19.7396
Epoch 4/10
1378/1378 [==============================] - 8s 6ms/step - loss: 15.5845 - mae: 1.7656 - mse: 15.5845 - val_loss: 20.7613 - val_mae: 2.2173 - val_mse: 20.7613
Epoch 5/10
1378/1378 [==============================] - 8s 5ms/step - loss: 13.9901 - mae: 1.6387 - mse: 13.9901 - val_loss: 15.8348 - val_mae: 1.6397 - val_mse: 15.8348
Epoch 6/10
1378/1378 [==============================] - 8s 6ms/step - loss: 12.7895 - mae: 1.5403 - mse: 12.7895 - val_loss: 19.9069 - val_mae: 1.94

In [160]:
from sklearn.metrics import mean_squared_error

test_predictions = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, test_predictions)
print('mse={0}'.format(mse))
predict_df = pd.DataFrame(test_predictions)

display(predict_df)

431/431 [==============================] - 1s 2ms/step
mse=107.10431104540386


,0
0,145.257462
1,8.720414
2,47.618790
3,38.239334
4,25.156794
...,...
13769,54.568874
13770,40.176113
13771,169.835861
13772,23.798340


#### why add tokens word2vec lower the performance?

1. currently vsize=10, which is too small
2. add wor2vec -> too many features -> too many model paras -> complex to train
- bigger epoch